# LangChain-RAG练习

参考教程：https://docs.langchain.com/oss/python/langchain/rag#ollama


In [13]:
import bs4
from langchain.agents import create_agent
from langchain_community.document_loaders import WebBaseLoader
from langchain.messages import MessageLikeRepresentation
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)



In [14]:

# 创建Embedding模型

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")

# 初始化向量存储
vector_store = InMemoryVectorStore(embeddings)
# Index chunks
_ = vector_store.add_documents(documents=all_splits)



In [20]:
from langchain_community.tools import Tool
from langchain.tools import tool

# @tool
# def get_current_time(query: str) -> str:
#     """Get the current time. Use when user asks about time or date."""
#     from datetime import datetime
#     return f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

# # Then use it in your tools list
# tools = [get_current_time]

# Construct a tool for retrieving context
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)



In [30]:

# pip install -U langchain-ollama
from langchain_ollama import ChatOllama


model = ChatOllama(model='deepseek-r1')
agent = create_agent(model, tools, system_prompt=prompt)


# from langchain.agents import initialize_agent, AgentType
# from langchain.tools import Tool
# from langchain_community.llms import Ollama

# llm = Ollama(model="llama3:latest")

# # Define tools (same as before)

# agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# result = agent.run("What time is it?")
# print(result)

In [32]:
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.


ResponseError: registry.ollama.ai/library/deepseek-r1:latest does not support tools (status code: 400)